# Dictionaries example
For a more in-depth explanation read
https://www.gurobi.com/documentation/9.0/quickstart_windows/py_python_dictionary_examp.html

In [48]:
import gurobipy as gp
from gurobipy import GRB

## Custom gurobipy classes

#### The `multidict` class

The `multidict` class allows to initialize multiple dictionaries at once.
It takes as an input a dictionary in which each entry's value is a list of length n. The multidict class will then split these lists, creating n seperate dictionaries that share the same keys.

In [49]:
names, lower, upper = gp.multidict(dict(x=[0,1], y=[1,2], z=[0,3]))
print(names)
print(lower)
print(upper)

['x', 'y', 'z']
{'x': 0, 'y': 1, 'z': 0}
{'x': 1, 'y': 2, 'z': 3}


#### The `tuplelist` class

The `tuplelist` class stores a list of tuples and is a subclass of the standard list class. The .select function is an efficient way to extract specific tuples based on the given criteria. It works with wildcards ('*') and multi-criteria inputs (lists)

In [50]:
l = gp.tuplelist([(1, 2), (1, 3), (2, 3), (2, 4)])
l.select(1, '*')  # will give all entries that have 1 in the first position

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
>

In [51]:
l.select('*', [2, 4])  # will return all entries that have 2 or 4 at the second position

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 2 , 4 )
>

#### The `tupledict` class

The `tupledict` class is a subclass of the python dictionaries that uses `tuplelist` as keys (so the .select method can be used to select subsets of the `tupledict`). In a tupledict, Gurobi decision variables can be stored that correspond to the tuple key. In a VRP, e.g., the key tuple may be (node i, node j, vehicle v) and the tupledict class allows to easily select only those arcs that originate from node i. 

In [52]:
l = list([(1, 2), (1, 3), (2, 3), (2, 4)])
m = gp.Model(name='dict1')
d = m.addVars(l, name='d')  # the addVars method creates Variables and returns them as a tupledict
m.update()
type(d)

gurobipy.tupledict

In [53]:
d  # all the variables in the dictionary with their names

{(1, 2): <gurobi.Var d[1,2]>,
 (1, 3): <gurobi.Var d[1,3]>,
 (2, 3): <gurobi.Var d[2,3]>,
 (2, 4): <gurobi.Var d[2,4]>}

In [54]:
sum(d.select(1, '*'))  # the tupledict can be used to create linear expressions of the variables

<gurobi.LinExpr: d[1,2] + d[1,3]>

In [55]:
d.sum(1, '*')  # is the faster way of doing the same

<gurobi.LinExpr: d[1,2] + d[1,3]>

In [56]:
coeff = {(1,2):5, (2,3):7}
# coeff = {l[0]: 5, l[2]:7}
d.prod(coeff)   # the .prod method of the tupledict class handles expressions where variable coefficients are not 1
                # the input dictionary has the same tuple keys as the variables themselves

<gurobi.LinExpr: 5.0 d[1,2] + 7.0 d[2,3]>

In [57]:
d.prod(coeff, 2)  # this method also accepts a filter

<gurobi.LinExpr: 7.0 d[2,3]>

In [58]:
m.getVars()

[<gurobi.Var d[1,2]>,
 <gurobi.Var d[1,3]>,
 <gurobi.Var d[2,3]>,
 <gurobi.Var d[2,4]>]

## netflow.py example
Two commodities (Pencils and Pens) are produced in two cities (Detroit and Denver), and must be shipped to warehouses in three cities (Boston, New York, and Seattle) to satisfy given demand. Each arc in the transportation network has a cost associated with it, and a total capacity.
The objective is to minimize the sum of the arc transportation costs

In [59]:
# Base Data
commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

arcs, capacity = gp.multidict({
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'New York'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'New York'): 120,
    ('Denver',  'Seattle'):  120
})
arcs, capacity

(<gurobi.tuplelist (6 tuples, 2 values each):
  ( Detroit , Boston   )
  ( Detroit , New York )
  ( Detroit , Seattle  )
  ( Denver  , Boston   )
  ( Denver  , New York )
  ( Denver  , Seattle  )
 >,
 {('Detroit', 'Boston'): 100,
  ('Detroit', 'New York'): 80,
  ('Detroit', 'Seattle'): 120,
  ('Denver', 'Boston'): 120,
  ('Denver', 'New York'): 120,
  ('Denver', 'Seattle'): 120})

In [60]:
# Cost for commodity-source-destination
cost = {
    ('Pencils', 'Detroit', 'Boston'):   10,
    ('Pencils', 'Detroit', 'New York'): 20,
    ('Pencils', 'Detroit', 'Seattle'):  60,
    ('Pencils', 'Denver',  'Boston'):   40,
    ('Pencils', 'Denver',  'New York'): 40,
    ('Pencils', 'Denver',  'Seattle'):  30,
    ('Pens',    'Detroit', 'Boston'):   20,
    ('Pens',    'Detroit', 'New York'): 20,
    ('Pens',    'Detroit', 'Seattle'):  80,
    ('Pens',    'Denver',  'Boston'):   60,
    ('Pens',    'Denver',  'New York'): 70,
    ('Pens',    'Denver',  'Seattle'):  30}

# Demand
inflow = {
    ('Pencils', 'Detroit'):   50,  # Production site -> positive (supply)
    ('Pencils', 'Denver'):    60,
    ('Pencils', 'Boston'):   -50,  # warehouse -> negative (demand)
    ('Pencils', 'New York'): -50,
    ('Pencils', 'Seattle'):  -10,
    ('Pens',    'Detroit'):   60,
    ('Pens',    'Denver'):    40,
    ('Pens',    'Boston'):   -40,
    ('Pens',    'New York'): -30,
    ('Pens',    'Seattle'):  -30}

In [62]:
# Optimization modes
m_netflow = gp.Model('netflow')

In [63]:
# Add variables in bulk, indexed by integers or lists
flow = m_netflow.addVars(commodities, arcs, obj=cost, name='flow')
m_netflow.update()
flow

{('Pencils', 'Detroit', 'Boston'): <gurobi.Var flow[Pencils,Detroit,Boston]>,
 ('Pencils',
  'Detroit',
  'New York'): <gurobi.Var flow[Pencils,Detroit,New York]>,
 ('Pencils', 'Detroit', 'Seattle'): <gurobi.Var flow[Pencils,Detroit,Seattle]>,
 ('Pencils', 'Denver', 'Boston'): <gurobi.Var flow[Pencils,Denver,Boston]>,
 ('Pencils', 'Denver', 'New York'): <gurobi.Var flow[Pencils,Denver,New York]>,
 ('Pencils', 'Denver', 'Seattle'): <gurobi.Var flow[Pencils,Denver,Seattle]>,
 ('Pens', 'Detroit', 'Boston'): <gurobi.Var flow[Pens,Detroit,Boston]>,
 ('Pens', 'Detroit', 'New York'): <gurobi.Var flow[Pens,Detroit,New York]>,
 ('Pens', 'Detroit', 'Seattle'): <gurobi.Var flow[Pens,Detroit,Seattle]>,
 ('Pens', 'Denver', 'Boston'): <gurobi.Var flow[Pens,Denver,Boston]>,
 ('Pens', 'Denver', 'New York'): <gurobi.Var flow[Pens,Denver,New York]>,
 ('Pens', 'Denver', 'Seattle'): <gurobi.Var flow[Pens,Denver,Seattle]>}

In [39]:
# Arc capacity constraints
m_netflow.addConstrs(
    (flow.sum('*', i, j) <= capacity[i, j] for i, j in arcs), 'cap'
) # uses python generators

{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [41]:
# flow conservation constraints
m_netflow.addConstrs(
    (flow.sum(h, '*', j) + inflow[h,j] == flow.sum(h, j, '*') 
     for h in commodities for j in nodes), 'node')

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [42]:
# optimize
m_netflow.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 22 rows, 12 columns and 48 nonzeros
Model fingerprint: 0xad4b30fe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 22 rows and 12 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra one simplex iteration after uncrush
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.03 seconds
Optimal objective  5.500000000e+03


In [46]:
# print solution
if m_netflow.status == GRB.OPTIMAL:
    solution = m_netflow.getAttr('x', flow)
    for h in commodities:
        print(f'\nOptimal flows for {h}')
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print(f'{i} -> {j}: {solution[h, i, j]}')


Optimal flows for Pencils
Detroit -> Boston: 50.0
Denver -> New York: 50.0
Denver -> Seattle: 10.0

Optimal flows for Pens
Detroit -> Boston: 30.0
Detroit -> New York: 30.0
Denver -> Boston: 10.0
Denver -> Seattle: 30.0
